### Preparation

In [1]:
import pandas as pd
import numpy as np
import json
import folium
from folium.plugins import BeautifyIcon
import openrouteservice as ors
from openrouteservice import client
import time
#!pip install folium
#!pip install openrouteservice

In [2]:
df = pd.read_csv('../data/raw_data/properties.csv')

In [4]:
df.head()

,name,cost_text,coordinates,type,bond,features,unverified_features,desc,costs,beds,baths,parking
0,8092Y/33 Rose Lane Melbourne VIC 3000,$60 per week,"[-37.8150137, 144.9538694]",Apartment / Unit / Flat,261.0,[],[],['. Close to Southern Cross Station. Available...,60.0,1,1,1
1,602/118 Franklin Street Melbourne VIC 3000,$330,"[-37.8082052, 144.9589035]",Apartment / Unit / Flat,1320.0,[],[],['The exclusive address simply speaks for itse...,330.0,1,1,0
2,1112/333-351 Exhibition Street Melbourne VIC 3000,$600 per week,"[-37.8078956, 144.9682873]",Apartment / Unit / Flat,2607.0,['Furnished'],[],['Positioned in this most favourable location ...,600.0,2,2,0
3,3002/288 Spencer St Melbourne VIC 3000,$510,"[-37.813775, 144.9520948]",Apartment / Unit / Flat,2216.0,[],[],['This fantastic apartment located on the 30th...,510.0,2,1,1
4,4/180 Little Collins Street Melbourne VIC 3000,$500 per week,"[-37.8138601, 144.9679067]",Apartment / Unit / Flat,2173.0,['Furnished'],[],['Enjoy what the renowned reputation The Georg...,500.0,1,1,0


#### Get the coordinates of MelbourneCBD from OpenStreetMap

In [14]:
coord_CBD = [-37.8152834, 144.9618789]  
Mel = folium.Map(location=coord_CBD,titles='MelbourneCBD',zoom_start = 8)

### Mark first 10 apartments in properties.csv

In [43]:
for i in range(10): 
    folium.Marker(
        location = json.loads(df['coordinates'][i]),
        icon = BeautifyIcon(
            icon_shape='marker',
            number=i,
            spin=True,
            text_color='blue',
        )
    ).add_to(Mel)

folium.Marker(
    location=coord_CBD,
    popup='Melbourne CBD',
    icon=folium.Icon(color="red", icon="info-sign")
).add_to(Mel)

In [44]:
Mel

### reverse coords for openrouteservice API

In [3]:
def fun(i):
    return json.loads(i)
df_new = df
df_new['lon'] = [fun(i)[1] for i in df['coordinates']]
df_new['lat'] = [fun(i)[0] for i in df['coordinates']]
df_new['revcoord'] = list(zip(df_new.lon,df_new.lat))
df_new['revcoord']

0        (144.9538694, -37.8150137)
1        (144.9589035, -37.8082052)
2        (144.9682873, -37.8078956)
3         (144.9520948, -37.813775)
4        (144.9679067, -37.8138601)
                    ...            
15231     (145.725239, -38.6340011)
15232    (145.7031282, -38.6412502)
15233    (145.7358082, -38.6287143)
15234    (145.7293638, -38.6314613)
15235     (145.717275, -38.6313719)
Name: revcoord, Length: 15236, dtype: object

### Calculate distances from first 100 apartments to MelbourneCBD(as travelled by car)

In [4]:

from openrouteservice.directions import directions
df_new['dis_to_melCBD'] = 0
rev_melcbd_coor = (144.9618789,-37.8152834)
client = ors.Client(key='5b3ce3597851110001cf6248e9e6879092a047d58005d1ea0a6bc7b5') # Specify your personal API key
for i in range(100):
    coords = (df_new['revcoord'][i],rev_melcbd_coor)
    routes = directions(client,coords, profile='driving-car') 
    df_new['dis_to_melCBD'][i] = routes['routes'][0]['summary']['distance']
    print('Distance from {name} to MelbourneCBD is {dis}: '.format(name=df_new['name'][i],dis=df_new['dis_to_melCBD'][i]))
    if(i%30==0 and i!=0 ): #
        time.sleep(60)


<ipython-input-4-88eaaa6a9dfd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['dis_to_melCBD'][i] = routes['routes'][0]['summary']['distance']


Distance from 8092Y/33 Rose Lane Melbourne VIC 3000 to MelbourneCBD is 1424: 
Distance from 602/118 Franklin Street Melbourne VIC 3000 to MelbourneCBD is 1241: 
Distance from 1112/333-351 Exhibition Street Melbourne VIC 3000 to MelbourneCBD is 1692: 
Distance from 3002/288 Spencer St Melbourne VIC 3000 to MelbourneCBD is 1612: 
Distance from 4/180 Little Collins Street Melbourne VIC 3000 to MelbourneCBD is 627: 
Distance from 1605/565 Flinders Street Melbourne VIC 3000 to MelbourneCBD is 1585: 
Distance from 612/408 Lonsdale Street Melbourne VIC 3000 to MelbourneCBD is 756: 
Distance from 2108/288 Spencer Street Melbourne VIC 3000 to MelbourneCBD is 1612: 
Distance from 1505/250 Elizabeth Street Melbourne VIC 3000 to MelbourneCBD is 531: 
Distance from 2205/22-24 Jane Bell Lane Melbourne VIC 3000 to MelbourneCBD is 1220: 
Distance from G11/61 MacKenzie Street Melbourne VIC 3000 to MelbourneCBD is 1431: 
Distance from 9/15 Rankins Lane Melbourne VIC 3000 to MelbourneCBD is 910: 
Distanc

### Find nearby schools, parks, supermarkets, cinemas for departments

#### Apartment isochrones within 5 mins car-driving

In [16]:
apartments = dict() # apartments location
for i in range(100):
    apartments[df_new['name'][i]]={'location':df_new['revcoord'][i]}
apartments

{'8092Y/33 Rose Lane Melbourne VIC 3000': {'location': (144.9538694,
   -37.8150137)},
 '602/118 Franklin Street Melbourne VIC 3000': {'location': (144.9589035,
   -37.8082052)},
 '1112/333-351 Exhibition Street Melbourne VIC 3000': {'location': (144.9682873,
   -37.8078956)},
 '3002/288 Spencer St Melbourne VIC 3000': {'location': (144.9520948,
   -37.813775)},
 '4/180 Little Collins Street Melbourne VIC 3000': {'location': (144.9679067,
   -37.8138601)},
 '1605/565 Flinders Street Melbourne VIC 3000': {'location': (144.9559072,
   -37.8210586)},
 '612/408 Lonsdale Street Melbourne VIC 3000': {'location': (144.9604012,
   -37.8125979)},
 '2108/288 Spencer Street Melbourne VIC 3000': {'location': (144.9520948,
   -37.813775)},
 '1505/250 Elizabeth Street Melbourne VIC 3000': {'location': (144.9628184,
   -37.812564)},
 '2205/22-24 Jane Bell Lane Melbourne VIC 3000': {'location': (144.9662652,
   -37.8102205)},
 'G11/61 MacKenzie Street Melbourne VIC 3000': {'location': (144.9663033,
  

In [26]:
from openrouteservice import client
ors = client.Client(key='5b3ce3597851110001cf624837cc3a9aa2654c29aa1341e88a025b98')
Mel2 = folium.Map(location=coord_CBD,titles='MelbourneCBD',zoom_start = 10)
# Take first 100 apartments as examples
# 300s == 5 mins
piso = {'profile': 'driving-car',
        'range': [300],  
        'attributes': ['total_pop']  
        }
i=0
for na, apartment in apartments.items():

        print("Name:{} \n".format(na))
        piso['locations'] = [apartment['location']]  # parameters
        apartment['iso'] = ors.isochrones(**piso)  # isochrones request
        folium.features.GeoJson(apartment['iso']).add_to(Mel2)  
        folium.map.Marker(list(reversed(apartment['location'])),  #folium coords are reversed from ordinary
                      icon=folium.Icon(color='red',
                                       icon='home',
                                       ),
                      popup=na,
                      ).add_to(Mel2)  # Add apartment isochrones to map
        i=i+1
        if(i%19==0):
                time.sleep(60)

Mel2

Name:8092Y/33 Rose Lane Melbourne VIC 3000 

Name:602/118 Franklin Street Melbourne VIC 3000 

Name:1112/333-351 Exhibition Street Melbourne VIC 3000 

Name:3002/288 Spencer St Melbourne VIC 3000 

Name:4/180 Little Collins Street Melbourne VIC 3000 

Name:1605/565 Flinders Street Melbourne VIC 3000 

Name:612/408 Lonsdale Street Melbourne VIC 3000 

Name:2108/288 Spencer Street Melbourne VIC 3000 

Name:1505/250 Elizabeth Street Melbourne VIC 3000 

Name:2205/22-24 Jane Bell Lane Melbourne VIC 3000 

Name:G11/61 MacKenzie Street Melbourne VIC 3000 

Name:9/15 Rankins Lane Melbourne VIC 3000 

Name:3510/442 Elizabeth St Melbourne VIC 3000 

Name:2004/288 Spencer Street Melbourne VIC 3000 

Name:2201/288 Spencer Street Melbourne VIC 3000 

Name:101/25-33 Wills Street Melbourne VIC 3000 

Name:2905B/11 Rose Lane Melbourne VIC 3000 

Name:1901/368 St Kilda Road Melbourne VIC 3000 

Name:1211/200 Spencer Street Melbourne VIC 3000 

Name:3/27 Flinders Lane Melbourne VIC 3000 

Name:102/9 Bl

#### Find schools, parks, supermarkets, cinemas in isochrones

In [28]:
params_poi = {'request': 'pois',
              'sortby': 'distance'}


categories_poi = {'school': [156],
                  'park': [280],
                  'supermarket': [518],
                  'cinema': [299] }

df_new ['school'] = 0
df_new['park'] = 0
df_new['supermarket'] = 0
df_new['cinema'] = 0
i=0
for na, apartment in apartments.items():
    apartment['categories'] = dict() 
    params_poi['geojson'] = apartment['iso']['features'][0]['geometry']
    print("{} apartment POIs nearby:\n".format(na))

    for typ, category in categories_poi.items():
        params_poi['filter_category_ids'] = category
        apartment['categories'][typ] = dict()
        apartment['categories'][typ]['geojson'] = ors.places(**params_poi)['features'] # POIs request 
        df_new[typ][i] = len(apartment['categories'][typ]['geojson'])
        print("\t{}: {}".format(typ,df_new[typ][i]))
    i=i+1
    if(i%20==0):
        time.sleep(30)

8092Y/33 Rose Lane Melbourne VIC 3000 apartment POIs nearby:



<ipython-input-28-4a56c14af9b3>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[typ][i] = len(apartment['categories'][typ]['geojson'])


	school: 11
	park: 170
	supermarket: 48
	cinema: 7
602/118 Franklin Street Melbourne VIC 3000 apartment POIs nearby:

	school: 16
	park: 166
	supermarket: 48
	cinema: 7
1112/333-351 Exhibition Street Melbourne VIC 3000 apartment POIs nearby:

	school: 19
	park: 173
	supermarket: 54
	cinema: 6
3002/288 Spencer St Melbourne VIC 3000 apartment POIs nearby:

	school: 10
	park: 108
	supermarket: 44
	cinema: 6
4/180 Little Collins Street Melbourne VIC 3000 apartment POIs nearby:

	school: 13
	park: 146
	supermarket: 49
	cinema: 7
1605/565 Flinders Street Melbourne VIC 3000 apartment POIs nearby:

	school: 10
	park: 130
	supermarket: 44
	cinema: 5
612/408 Lonsdale Street Melbourne VIC 3000 apartment POIs nearby:

	school: 16
	park: 176
	supermarket: 53
	cinema: 7
2108/288 Spencer Street Melbourne VIC 3000 apartment POIs nearby:

	school: 10
	park: 108
	supermarket: 44
	cinema: 6
1505/250 Elizabeth Street Melbourne VIC 3000 apartment POIs nearby:

	school: 15
	park: 175
	supermarket: 51
	cinem

ApiError: 403 ({'error': 'Quota exceeded'})

In [30]:
df_new.to_csv('../data/raw_data/new_properties.csv',index = False)